# 05 - Fine-Tuning LLMs for Compliance Classification

## Customize an LLM for Your Specific Use Case

**Goal:** Fine-tune a base LLM on your labeled compliance data to create a domain-specific model.

### Why Fine-Tune?

| Approach | Accuracy | Cost | Use Case |
|----------|----------|------|----------|
| **Base LLM** | Good | Higher | General questions |
| **Fine-tuned LLM** | Better | Lower (smaller model) | Specific classification tasks |

### What We Have

From `data/finetune_training.jsonl`:
- 500 labeled prompt/completion pairs
- 200 CLEAN examples
- 75 each of: INSIDER_TRADING, CONFIDENTIALITY_BREACH, PERSONAL_TRADING, INFO_BARRIER_VIOLATION

---

In [ ]:
# Setup
from snowflake.snowpark import Session

session = Session.builder.getOrCreate()
session.use_warehouse("COMPLIANCE_DEMO_WH")
session.use_database("COMPLIANCE_DEMO")
session.use_schema("ML")

print(f"Connected as: {session.get_current_user()}")

## 1. Review Training Data

Check our fine-tuning training samples loaded from the JSONL file.

In [ ]:
# Check training data
print("--- Fine-Tuning Training Data ---")
session.sql("SELECT COUNT(*) as TOTAL_SAMPLES FROM COMPLIANCE_DEMO.ML.FINETUNE_TRAINING").show()

# Preview a few samples
print("\n--- Sample Training Pairs ---")
session.sql("""
    SELECT 
        SAMPLE_ID,
        LEFT(PROMPT, 100) as PROMPT_PREVIEW,
        LEFT(COMPLETION, 100) as COMPLETION_PREVIEW
    FROM COMPLIANCE_DEMO.ML.FINETUNE_TRAINING
    LIMIT 5
""").show()

In [ ]:
# Check label distribution in training data
print("--- Label Distribution in Training Data ---")
session.sql("""
    SELECT 
        SPLIT_PART(COMPLETION, ' - ', 1) as LABEL,
        COUNT(*) as COUNT
    FROM COMPLIANCE_DEMO.ML.FINETUNE_TRAINING
    GROUP BY 1
    ORDER BY 2 DESC
""").show()

## 2. Start Fine-Tuning Job

Use Snowflake Cortex FINETUNE to create a compliance-specific model.

**Note:** Fine-tuning is an asynchronous job that can take 30min to several hours depending on data size.

In [ ]:
# Start fine-tuning job
# IMPORTANT: This is an async operation. The job will run in the background.
print("🚀 Starting fine-tuning job...")

result = session.sql("""
    SELECT SNOWFLAKE.CORTEX.FINETUNE(
        'CREATE',
        'compliance_classifier_v1',
        'mistral-7b',
        'SELECT PROMPT, COMPLETION FROM COMPLIANCE_DEMO.ML.FINETUNE_TRAINING'
    ) as JOB_INFO
""").collect()

print("Fine-tuning job submitted!")
print(result[0]["JOB_INFO"])

## 3. Check Job Status

Monitor the fine-tuning job progress.

In [ ]:
# Check job status
print("--- Fine-Tuning Job Status ---")
session.sql("""
    SELECT SNOWFLAKE.CORTEX.FINETUNE('DESCRIBE', 'compliance_classifier_v1') as STATUS
""").show()

## 4. Compare Base vs Fine-Tuned Model

Once the fine-tuning job completes, compare performance.

In [ ]:
# Compare base vs fine-tuned model (run after job completes)
# This cell will fail if the fine-tuning job hasn't completed yet

print("--- Base Model vs Fine-Tuned Comparison ---")
print("Testing on 5 sample emails...\n")

# Test prompt
test_prompt = '''Classify this hedge fund email for compliance violations.

Email: Hey, just got back from drinks with my contact at Acme Corp. 
They're announcing a major acquisition next week - not public yet. 
Might want to position ahead of the news. Keep this between us.

Classification:'''

print("Test Email:")
print("-" * 40)
print(test_prompt)
print("-" * 40)

# Compare responses
print("\n📊 BASE MODEL (mistral-7b):")
base_result = session.sql(f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-7b',
        '{test_prompt}'
    ) as RESPONSE
""").collect()
print(base_result[0]["RESPONSE"])

# Fine-tuned model (uncomment after job completes)
# print("\n🎯 FINE-TUNED MODEL (compliance_classifier_v1):")
# ft_result = session.sql(f"""
#     SELECT SNOWFLAKE.CORTEX.COMPLETE(
#         'compliance_classifier_v1',
#         '{test_prompt}'
#     ) as RESPONSE
# """).collect()
# print(ft_result[0]["RESPONSE"])

## 5. Production Deployment

How to use your fine-tuned model in production.

In [ ]:
# Production usage pattern
print('''
=================================================================
PRODUCTION USAGE: Fine-Tuned Compliance Classifier
=================================================================

1. BATCH CLASSIFICATION (for daily email processing):

   SELECT 
       EMAIL_ID,
       SUBJECT,
       SNOWFLAKE.CORTEX.COMPLETE(
           'compliance_classifier_v1',  -- Your fine-tuned model
           CONCAT(
               'Classify this hedge fund email for compliance violations. ',
               'Email: ', BODY,
               ' Classification:'
           )
       ) as CLASSIFICATION
   FROM EMAILS
   WHERE DATE(SENT_AT) = CURRENT_DATE();


2. REAL-TIME CLASSIFICATION (for email gateway integration):

   -- Create a UDF for cleaner syntax
   CREATE OR REPLACE FUNCTION classify_email(email_body STRING)
   RETURNS STRING
   AS $$
       SNOWFLAKE.CORTEX.COMPLETE(
           'compliance_classifier_v1',
           CONCAT('Classify this hedge fund email: ', email_body, ' Classification:')
       )
   $$;
   
   -- Then call:
   SELECT classify_email('Your email text here');


3. COST COMPARISON (estimated per 1000 emails):

   ┌─────────────────────────────────────────┐
   │ Model               │ Cost/1K │ Latency │
   ├─────────────────────────────────────────┤
   │ claude-opus-4-5           │ ~$10.00 │ ~200ms  │
   │ mistral-7b (base)   │ ~$0.50  │ ~50ms   │
   │ compliance_v1 (ft)  │ ~$0.50  │ ~50ms   │
   └─────────────────────────────────────────┘
   
   Fine-tuned models: Same cost as base, better accuracy!
''')

## Summary

**What we demonstrated:**
- Reviewed 500 labeled training samples for fine-tuning
- Started a Cortex FINETUNE job (async operation)
- Showed how to check job status and compare models
- Provided production deployment patterns

**Benefits of Fine-Tuning:**
1. **Better Accuracy:** Model learns your specific compliance terminology
2. **Lower Latency:** Smaller base model (mistral-7b) vs large models
3. **Same Cost:** Fine-tuned models cost the same as their base
4. **Consistent Output:** Trained on your exact output format

**Next:** In notebook 06, we'll use vector embeddings for semantic search.